### Fine-Tune Longformer Base

- Joel Stremmel
- 04-13-23

##### About

Fine-Tune Longformer Base on the formatted data using K-Fold Cross-Validation and save the scores.

##### Install Libraries

In [1]:
!pip install -q pdfminer.six
!pip install -q pandas
!pip install -q transformers
!pip install -q openpyxl
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 21.3 MB/s eta 0:00:00


##### Imports

In [2]:
import os
import re
import glob
import pickle
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

##### Connect to Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##### Set Parameters

In [4]:
max_seq_len = 4096
batch_size = 8
accumulation_steps = 4
lr = 0.00002
weight_decay = 0.01
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_epsilon = 0.00000001
warmup_steps = 5
logging_steps = 1
num_workers = 2
seed = 44
epochs = 10
fp16 = True
colab = True
require_high_ram = False
input_dir = "/content/drive/MyDrive/data/"
model_output_dir = "model_output"
results_dir = "/content/drive/MyDrive/results/"
model_key = "lfb"
lm_path = "allenai/longformer-base-4096"

##### Disable Tokenizer Parallelism
This is mostly to avoid warnings.

In [5]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

##### Check Runtime

In [6]:
if colab:
  
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)

if require_high_ram:

    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Thu Apr 13 19:52:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##### Load Formatted Data

In [7]:
with open(os.path.join(input_dir, 'X_folds.pkl'), 'rb') as f:
    X_folds = pickle.load(f)

with open(os.path.join(input_dir, 'y_folds.pkl'), 'rb') as f:
    y_folds = pickle.load(f)

##### Check Data Shape

In [8]:
assert len(X_folds) == len(y_folds), "Expected the same number of folds in X and y."
X = list(X_folds.values())
y = list(y_folds.values())

##### Check Target Prevalence

In [9]:
print(f"Target prevalance: {np.mean(np.concatenate(y))}.")

Target prevalance: 0.5.


##### Check that GPU is Available

In [10]:
assert torch.cuda.is_available(), "Run this script on a GPU."
print(torch.__version__)

2.0.0+cu118


##### Tokenize Text and Fit Model to Each Fold

In [11]:
y_probs, y_trues = [], []
for i in range(len(X)):
    
    print(f"Fitting model using fold {i} as out of fold data.")
    
    # Identify train folds and shuffle samples
    X_train, y_train = np.concatenate(X[0:i] + X[i+1:], axis=0), np.concatenate(y[0:i] + y[i+1:], axis=0)
    indices = np.arange(len(y_train))
    np.random.shuffle(indices)
    X_train, y_train = X_train[indices], y_train[indices]
    
    # Identify test folds
    X_test, y_test = X[i], y[i]
    
    # Format text and label data as HuggingFace dataset
    train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
    test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})
    
    # Load model and tokenizer
    # This will reset the model weights with each new iteration
    # To Do: Test adding a special token to separate utterances!
    tokenizer = AutoTokenizer.from_pretrained(lm_path)
    model = AutoModelForSequenceClassification.from_pretrained(
        lm_path,
        num_labels=2,
        return_dict=True,
        problem_type="single_label_classification"
    )
    
    # Define function to tokenize text
    def tokenize_function(batch):
        
        return tokenizer(
            batch["text"],
            padding="max_length",
            truncation=True,
            max_length=max_seq_len
        )
    
    # Tokenize train dataset
    train_dataset = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
        batch_size=batch_size
    )
    train_dataset.set_format("pt")
    
    # Tokenize test dataset
    test_dataset = test_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
        batch_size=batch_size
    )
    test_dataset.set_format("pt")
    
    # Define training arguments
    training_args= TrainingArguments(
        output_dir=model_output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=accumulation_steps,
        warmup_steps=warmup_steps,
        logging_steps=logging_steps,
        weight_decay=weight_decay,
        learning_rate=lr,
        seed=seed,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        dataloader_num_workers=num_workers,
        fp16=fp16,
        run_name=model_key,
        logging_strategy="steps",
        save_strategy="no",
        lr_scheduler_type='linear',
        optim="adamw_torch",
        do_eval=False,
        fp16_full_eval=False,
        sharded_ddp=False,
        gradient_checkpointing=True,
        load_best_model_at_end=True,
        prediction_loss_only=False,
        disable_tqdm=True,
        logging_dir=None,
    )
    
    # Define model training
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset
    )
    
    # Train model
    trainer.train()
    
    # Predict on test dataset
    output = trainer.predict(test_dataset)
    labels = output.label_ids
    y_prob = torch.sigmoid(torch.tensor(output.predictions).double()).numpy()[:, 1]

    # Save scores and labels
    y_probs.append(y_prob)
    y_trues.append(labels)

Fitting model using fold 0 as out of fold data.


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight'

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

{'loss': 0.7289, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.57}
{'loss': 0.7252, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.14}
{'loss': 0.7465, 'learning_rate': 1.2e-05, 'epoch': 1.71}
{'loss': 0.7599, 'learning_rate': 1.6000000000000003e-05, 'epoch': 2.29}
{'loss': 0.6967, 'learning_rate': 2e-05, 'epoch': 2.86}
{'loss': 0.7078, 'learning_rate': 1.6000000000000003e-05, 'epoch': 3.43}
{'loss': 0.713, 'learning_rate': 1.2e-05, 'epoch': 4.0}
{'loss': 0.676, 'learning_rate': 8.000000000000001e-06, 'epoch': 4.57}
{'loss': 0.6783, 'learning_rate': 4.000000000000001e-06, 'epoch': 5.14}
{'loss': 0.6542, 'learning_rate': 0.0, 'epoch': 5.71}
{'train_runtime': 111.6717, 'train_samples_per_second': 4.836, 'train_steps_per_second': 0.09, 'train_loss': 0.708643090724945, 'epoch': 5.71}
Fitting model using fold 1 as out of fold data.


...

{'loss': 0.6907, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.5}
{'loss': 0.6792, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.0}
{'loss': 0.7044, 'learning_rate': 1.2e-05, 'epoch': 1.5}
{'loss': 0.6882, 'learning_rate': 1.6000000000000003e-05, 'epoch': 2.0}
{'loss': 0.6898, 'learning_rate': 2e-05, 'epoch': 2.5}
{'loss': 0.6898, 'learning_rate': 1.866666666666667e-05, 'epoch': 3.0}
{'loss': 0.6721, 'learning_rate': 1.7333333333333336e-05, 'epoch': 3.5}
{'loss': 0.6874, 'learning_rate': 1.6000000000000003e-05, 'epoch': 4.0}
{'loss': 0.6523, 'learning_rate': 1.4666666666666666e-05, 'epoch': 4.5}
{'loss': 0.6687, 'learning_rate': 1.3333333333333333e-05, 'epoch': 5.0}
{'loss': 0.6766, 'learning_rate': 1.2e-05, 'epoch': 5.5}
{'loss': 0.6498, 'learning_rate': 1.0666666666666667e-05, 'epoch': 6.0}
{'loss': 0.6536, 'learning_rate': 9.333333333333334e-06, 'epoch': 6.5}
{'loss': 0.6281, 'learning_rate': 8.000000000000001e-06, 'epoch': 7.0}
{'loss': 0.5885, 'learning_rate': 6.66666666

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight'

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

{'loss': 0.6972, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.5}
{'loss': 0.6915, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.0}
{'loss': 0.6898, 'learning_rate': 1.2e-05, 'epoch': 1.5}
{'loss': 0.6855, 'learning_rate': 1.6000000000000003e-05, 'epoch': 2.0}
{'loss': 0.6874, 'learning_rate': 2e-05, 'epoch': 2.5}
{'loss': 0.6869, 'learning_rate': 1.866666666666667e-05, 'epoch': 3.0}
{'loss': 0.6585, 'learning_rate': 1.7333333333333336e-05, 'epoch': 3.5}
{'loss': 0.6954, 'learning_rate': 1.6000000000000003e-05, 'epoch': 4.0}
{'loss': 0.657, 'learning_rate': 1.4666666666666666e-05, 'epoch': 4.5}
{'loss': 0.6569, 'learning_rate': 1.3333333333333333e-05, 'epoch': 5.0}
{'loss': 0.6547, 'learning_rate': 1.2e-05, 'epoch': 5.5}
{'loss': 0.6588, 'learning_rate': 1.0666666666666667e-05, 'epoch': 6.0}
{'loss': 0.6374, 'learning_rate': 9.333333333333334e-06, 'epoch': 6.5}
{'loss': 0.6555, 'learning_rate': 8.000000000000001e-06, 'epoch': 7.0}
{'loss': 0.6076, 'learning_rate': 6.666666666

##### Save Model Probabilities on Test Folds and True Labels

In [12]:
with open(os.path.join(results_dir, f'{model_key}_y_trues.pkl'), 'wb') as f:
    pickle.dump(y_trues, f)

with open(os.path.join(results_dir, f'{model_key}_y_probs.pkl'), 'wb') as f:
    pickle.dump(y_probs, f)